In [23]:
import pandas as pd
import numpy as np
data_path = "./csv/"

patients = pd.read_csv(f"{data_path}patients.csv")
encounters = pd.read_csv(f"{data_path}encounters.csv")
conditions = pd.read_csv(f"{data_path}conditions.csv")
observations = pd.read_csv(f"{data_path}observations.csv")
medications = pd.read_csv(f"{data_path}medications.csv")
procedures = pd.read_csv(f"{data_path}procedures.csv")

# Convert dates (force all to tz-naive)
patients["BIRTHDATE"]  = pd.to_datetime(patients["BIRTHDATE"], errors="coerce").dt.tz_localize(None)
patients["DEATHDATE"]  = pd.to_datetime(patients["DEATHDATE"], errors="coerce").dt.tz_localize(None)
encounters["START"]    = pd.to_datetime(encounters["START"], errors="coerce").dt.tz_localize(None)

# Last encounter date
last_enc = encounters.groupby("PATIENT")["START"].max().reset_index()
last_enc.columns = ["Id", "last_encounter"]

patients = patients.merge(last_enc, on="Id", how="left")

# Build ENDDATE
patients["ENDDATE"] = patients["DEATHDATE"]
patients["ENDDATE"] = patients["ENDDATE"].fillna(patients["last_encounter"])
patients["ENDDATE"] = patients["ENDDATE"].fillna(pd.to_datetime("2025-11-01"))

# Remove timezone again (in case merge introduced object column)
patients["ENDDATE"] = pd.to_datetime(patients["ENDDATE"], errors="coerce").dt.tz_localize(None)

# Fix missing BIRTHDATE
patients["BIRTHDATE"] = patients["BIRTHDATE"].fillna(pd.to_datetime("1970-01-01"))

# Compute age safely
patients["age"] = ((patients["ENDDATE"] - patients["BIRTHDATE"]).dt.days / 365.25).astype(int)

# Gender
patients["is_male"] = (patients["GENDER"] == "M").astype(int)
patients["is_female"] = (patients["GENDER"] == "F").astype(int)

In [24]:
# Encounters
encounter_counts = encounters.groupby("PATIENT").size().reset_index(name="encounter_count")

encounters["TYPE"] = encounters["ENCOUNTERCLASS"].astype(str).str.lower()

encounter_types = encounters.groupby("PATIENT")["TYPE"].agg(list).reset_index()

def count_type(lst, t):
    return sum(1 for x in lst if t in x)

encounter_types["er_visits"] = encounter_types["TYPE"].apply(lambda x: count_type(x, "emergency"))
encounter_types["inpatient_visits"] = encounter_types["TYPE"].apply(lambda x: count_type(x, "inpatient"))
encounter_types["ambulatory_visits"] = encounter_types["TYPE"].apply(lambda x: count_type(x, "ambulatory"))

In [25]:
# Conditions
conditions["DESCRIPTION"] = conditions["DESCRIPTION"].astype(str).str.lower()

conditions["diabetes"] = conditions["DESCRIPTION"].str.contains("diabetes")
conditions["hypertension"] = conditions["DESCRIPTION"].str.contains("hypertens")
conditions["hyperlipidemia"] = conditions["DESCRIPTION"].str.contains("hyperlipid|cholesterol")
conditions["obesity"] = conditions["DESCRIPTION"].str.contains("obesity")
conditions["cancer"] = conditions["DESCRIPTION"].str.contains("cancer|neoplasm")
conditions["heart_failure"] = conditions["DESCRIPTION"].str.contains("heart failure|congestive")

chronic_conditions = (
    conditions.groupby("PATIENT")[
        ["diabetes","hypertension","hyperlipidemia","obesity","cancer","heart_failure"]
    ].any()
    .reset_index()
)

In [26]:
# Count total medications
med_counts = medications.groupby("PATIENT").size().reset_index(name="medication_count")

# Identify chronic medications (very important clinically)
medications["DESCRIPTION"] = medications["DESCRIPTION"].astype(str).str.lower()

medications["insulin"] = medications["DESCRIPTION"].str.contains("insulin")
medications["metformin"] = medications["DESCRIPTION"].str.contains("metformin")
medications["statin"] = medications["DESCRIPTION"].str.contains("statin")
medications["ace_inhibitor"] = medications["DESCRIPTION"].str.contains("pril")
medications["anticoagulant"] = medications["DESCRIPTION"].str.contains("warfarin|heparin|xaban")

med_chronic = (
    medications.groupby("PATIENT")[
        ["insulin","metformin","statin","ace_inhibitor","anticoagulant"]
    ].any()
    .reset_index()
)

In [27]:
observations["DESCRIPTION"] = observations["DESCRIPTION"].astype(str).str.lower()

# Key lab/vital signals
obs_keywords = ["glucose", "pressure", "oxygen", "bmi", "cholesterol"]

observations["is_relevant"] = observations["DESCRIPTION"].apply(
    lambda x: any(k in x for k in obs_keywords)
)

relevant_obs = observations[observations["is_relevant"]]

# lab count feature
lab_counts = (
    relevant_obs.groupby("PATIENT").size().reset_index(name="lab_measurement_count")
)

In [28]:
# Procedures
procedures["DESCRIPTION"] = procedures["DESCRIPTION"].astype(str).str.lower()

procedures["major_surgery"] = procedures["DESCRIPTION"].str.contains(
    "transplant|bypass|replacement|cabg|amputation|resection"
)

proc_features = (
    procedures.groupby("PATIENT")["major_surgery"].sum().reset_index(name="major_surgery_count")
)

In [34]:
patients["mortality"] = patients["DEATHDATE"].notna().astype(int)
df = patients[["Id", "age", "is_male", "is_female", "mortality"]].copy()

df = df.merge(
    encounter_counts[["PATIENT", "encounter_count"]],
    left_on="Id", right_on="PATIENT", how="left"
).drop(columns=["PATIENT"])

df = df.merge(
    encounter_types[["PATIENT", "er_visits", "inpatient_visits", "ambulatory_visits"]],
    left_on="Id", right_on="PATIENT", how="left"
).drop(columns=["PATIENT"])

df = df.merge(
    chronic_conditions,
    left_on="Id", right_on="PATIENT", how="left"
).drop(columns=["PATIENT"])

df = df.merge(
    med_counts,
    left_on="Id", right_on="PATIENT", how="left"
).drop(columns=["PATIENT"])

df = df.merge(
    med_chronic,
    left_on="Id", right_on="PATIENT", how="left"
).drop(columns=["PATIENT"])

df = df.merge(
    lab_counts,
    left_on="Id", right_on="PATIENT", how="left"
).drop(columns=["PATIENT"])

df = df.merge(
    proc_features,
    left_on="Id", right_on="PATIENT", how="left"
).drop(columns=["PATIENT"])

df = df.fillna(0)

feature_cols = [
    "age","is_male","is_female",
    "encounter_count","er_visits","inpatient_visits","ambulatory_visits",
    "diabetes","hypertension","hyperlipidemia","obesity","cancer","heart_failure",
    "medication_count","insulin","metformin","statin","ace_inhibitor","anticoagulant",
    "lab_measurement_count","major_surgery_count"
]

final_df = df[["Id"] + feature_cols + ["mortality"]].copy()

final_df.to_csv("final_df.csv", index=False, encoding="utf-8-sig")